In [1]:
import torch, random, numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # si usas multi-GPU
np.random.seed(SEED)
random.seed(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,)) #this is like the scaler of tensorflow
])


train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./data", train=True, download=True, transform=transform), #we use
    batch_size=64,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./data", train=False, download=True, transform=transform),
    batch_size=1000,
    shuffle=False
)

In [4]:
def train(model, train_loader, device, metric, epochs, optimizer):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):

        data, targets = data.to(device), targets.to(device)

        optimizer.zero_grad()

        output = model(data)

        loss = metric(output, targets)

        loss.backward()

        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Época {epoch} | Lote {batch_idx} | Pérdida: {loss.item():.4f}")

In [5]:
def evaluate(model, device, test_loader, metric):

    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(test_loader):
            data, targets = data.to(device), targets.to(device)

            output = model(data)

            test_loss += metric(output, targets).item()

            pred = output.argmax(dim=1, keepdim=True)

            correct += pred.eq(targets.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        accuracy = 100 * correct / len(test_loader.dataset)

    print(f"\nPérdida promedio en test: {test_loss:.4f}, Precisión: {accuracy:.2f}%\n")
    return test_loss, accuracy


In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,3,1)
        self.conv2 = nn.Conv2d(32,64,3,1)
        self.conv3 = nn.Conv2d(64,128,3,1)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

        self.fc1 = nn.Linear(128*6*6,256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256,10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x,2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x,2)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [7]:
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.005):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float("inf")
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            return

        # mejora relativa (en %)
        improvement = (self.best_loss - val_loss) / self.best_loss

        if improvement < self.min_delta:  # no mejora suficiente
            self.counter += 1
            print(f"⚠️ Sin mejora suficiente ({improvement:.4f}), paciencia {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
metric = nn.CrossEntropyLoss()

In [10]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [14]:
patience = 3
early_stopping = EarlyStopping(patience=patience, min_delta=0.005)
epochs = 25

for epoch in range(1, epochs + 1):
    train(model, train_loader, device, metric, epoch, optimizer)
    val_loss, accuracy = evaluate(model, device, test_loader, metric)

    if val_loss < early_stopping.best_loss or early_stopping.best_loss is None:
        torch.save(model.state_dict(), "best_model.pth")
        print(f"💾 Modelo guardado en epoch {epoch}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("⏹️ Early stopping activado en epoch", epoch)
        break

    scheduler.step()


Época 1 | Lote 0 | Pérdida: 2.3861
Época 1 | Lote 100 | Pérdida: 1.9069
Época 1 | Lote 200 | Pérdida: 1.4481
Época 1 | Lote 300 | Pérdida: 1.6375
Época 1 | Lote 400 | Pérdida: 1.4456
Época 1 | Lote 500 | Pérdida: 1.2556
Época 1 | Lote 600 | Pérdida: 1.2584
Época 1 | Lote 700 | Pérdida: 1.3103

Pérdida promedio en test: 0.0012, Precisión: 54.38%

💾 Modelo guardado en epoch 1
Época 2 | Lote 0 | Pérdida: 1.5691
Época 2 | Lote 100 | Pérdida: 1.3889
Época 2 | Lote 200 | Pérdida: 1.1068
Época 2 | Lote 300 | Pérdida: 1.3749
Época 2 | Lote 400 | Pérdida: 1.3255
Época 2 | Lote 500 | Pérdida: 1.3454
Época 2 | Lote 600 | Pérdida: 1.2531
Época 2 | Lote 700 | Pérdida: 0.8865

Pérdida promedio en test: 0.0012, Precisión: 60.99%

💾 Modelo guardado en epoch 2
Época 3 | Lote 0 | Pérdida: 0.8677
Época 3 | Lote 100 | Pérdida: 0.8881



KeyboardInterrupt



In [ ]:
# Guardar pesos
torch.save(model.state_dict(), "cifar10_good_acc_2.pth")

# Cargar pesos después
model = SimpleCNN()
model.load_state_dict(torch.load("cifar10_good_acc_2.pth"))
model.eval()


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4608, out_features=256, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

Im gonna make data augmentation to increase the accuracy


In [11]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) #this is like the scaler of tensorflow
])

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(32, scale=(0.8, 1.0)), # recorte y reescalado
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # cambios de color
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./data", train=True, download=True, transform=train_transform), #we use
    batch_size=64,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./data", train=False, download=True, transform=test_transform),
    batch_size=1000,
    shuffle=False
)

In [ ]:
patience = 5
early_stopping = EarlyStopping(patience=patience, min_delta=0.005)
epochs = 25

for epoch in range(1, epochs + 1):
    train(model, train_loader, device, metric, epoch, optimizer)
    val_loss, accuracy = evaluate(model, device, test_loader, metric)

    if val_loss < early_stopping.best_loss or early_stopping.best_loss is None:
        torch.save(model.state_dict(), "best_model.pth")
        print(f"💾 Modelo guardado en epoch {epoch}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("⏹️ Early stopping activado en epoch", epoch)
        break


Época 1 | Lote 0 | Pérdida: 0.9584
Época 1 | Lote 100 | Pérdida: 1.0312
Época 1 | Lote 200 | Pérdida: 1.2296
Época 1 | Lote 300 | Pérdida: 0.9972
Época 1 | Lote 400 | Pérdida: 1.0445
Época 1 | Lote 500 | Pérdida: 1.1165
Época 1 | Lote 600 | Pérdida: 1.0716
Época 1 | Lote 700 | Pérdida: 1.3136

Pérdida promedio en test: 0.0008, Precisión: 72.72%

💾 Modelo guardado en epoch 1
Época 2 | Lote 0 | Pérdida: 0.9829
Época 2 | Lote 100 | Pérdida: 1.1654
Época 2 | Lote 200 | Pérdida: 0.8735
Época 2 | Lote 300 | Pérdida: 0.7032
Época 2 | Lote 400 | Pérdida: 0.6932
Época 2 | Lote 500 | Pérdida: 0.7333
Época 2 | Lote 600 | Pérdida: 0.9427
Época 2 | Lote 700 | Pérdida: 0.7130

Pérdida promedio en test: 0.0007, Precisión: 75.19%

💾 Modelo guardado en epoch 2
Época 3 | Lote 0 | Pérdida: 0.7077
Época 3 | Lote 100 | Pérdida: 1.1094
Época 3 | Lote 200 | Pérdida: 1.1033
Época 3 | Lote 300 | Pérdida: 0.8736
Época 3 | Lote 400 | Pérdida: 0.8244
Época 3 | Lote 500 | Pérdida: 0.7462
Época 3 | Lote 600 | Pérdi

In [ ]:
# Guardar pesos
torch.save(model.state_dict(), "cifar10_good_acc_3.pth")

# Cargar pesos después
model = SimpleCNN()
model.load_state_dict(torch.load("cifar10_good_acc_3.pth"))
model.eval()


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4608, out_features=256, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
patience = 5
early_stopping = EarlyStopping(patience=patience, min_delta=0.005)
epochs = 25

for epoch in range(1, epochs + 1):
    train(model, train_loader, device, metric, epoch, optimizer)
    val_loss, accuracy = evaluate(model, device, test_loader, metric)

    if val_loss < early_stopping.best_loss or early_stopping.best_loss is None:
        torch.save(model.state_dict(), "best_model.pth")
        print(f"💾 Modelo guardado en epoch {epoch}")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("⏹️ Early stopping activado en epoch", epoch)
        break

    scheduler.step()


Época 1 | Lote 0 | Pérdida: 2.4044
Época 1 | Lote 100 | Pérdida: 1.8234
Época 1 | Lote 200 | Pérdida: 1.5376
Época 1 | Lote 300 | Pérdida: 1.7701
Época 1 | Lote 400 | Pérdida: 1.8023
Época 1 | Lote 500 | Pérdida: 1.5325
Época 1 | Lote 600 | Pérdida: 1.5201


To potentially increase the accuracy of your model, you could consider the following:

*   **Hyperparameter Tuning:** Experiment with different learning rates for the optimizer, different batch sizes, and more/fewer epochs if not using early stopping.
*   **Model Architecture:** Try a more complex CNN architecture, such as ResNet, VGG, or DenseNet, which are known to perform well on image classification tasks.
*   **More Data Augmentation:** Explore additional data augmentation techniques like Cutout, Mixup, or CutMix.
*   **Regularization:** Add L1 or L2 regularization to the model weights to prevent overfitting.
*   **Learning Rate Scheduling:** Implement a learning rate scheduler that reduces the learning rate during training.
*   **Ensembling:** Train multiple models and combine their predictions.
*   **Transfer Learning:** Use a pre-trained model on a large dataset (like ImageNet) and fine-tune it on your CIFAR-10 dataset.